In [1]:
import requests
import time as t
from mpl_toolkits.basemap import Basemap
from shapely.geometry import LineString
from matplotlib import pyplot as plt
import numpy as np
import os
import urllib
import json
%matplotlib inline

# Env vars and functions

In [2]:
apiKey = os.environ.get('MAPZEN_API')

def decode(encoded):
    inv = 1.0 / 1e6
    decoded = []
    previous = [0,0]
    i = 0
    while i < len(encoded):
        ll = [0,0]
        for j in [0, 1]:
            shift = 0
            byte = 0x20
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
            previous[j] = ll[j]
        decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    return decoded


def synthesize_gps(edges, shape, distribution="normal", stddev=None,uuid='999999'):
    
    jsonDict = {"uuid":"100343", "trace":[]}
    coords = decode(shape)
    sttm = t.time() - 100000
    
    for i, edge in enumerate(edges):
        dist = edge['length']
        speed = edge['speed']
        beginShapeIndex = edge['begin_shape_index']
        endShapeIndex = edge['end_shape_index']
        lon, lat = coords[endShapeIndex]
        if stddev is not None:
            avgLat = np.mean(np.array(coords)[:,1])
            stddevLon = stddev / 111.111     # approx. 111.111 km per deg lon unless very close to the poles
            stddevLat = stddev / (111.111 * np.cos(avgLat))     # approx 111.111 km * cos(lat) per deg lat
            lon += np.random.normal(scale=stddevLon)
            lat += np.random.normal(scale=stddevLat)
        dur = dist / speed * 3600.0
        time = sttm + dur
        time = int(round(time))
        if i == 0:
            st_lon, st_lat = coords[beginShapeIndex]
            jsonDict["trace"].append({"lat":st_lat,"lon":st_lon,"time":time})
        jsonDict["trace"].append({"lat": lat,"lon":lon,"time":time})
        sttm = time

    return jsonDict

# Open Traffic Reporter Validation Service 

### 1. Get New Route

In [94]:
# 349 38th St Oakland to 111 New Montgomery SF
stLat = 37.8261
stLon = -122.25872
endLat = 37.790168
endLon = -122.402264

jsonDict = {"locations":[
        {"lat":stLat,"lon":stLon,"type":"break"},
        {"lat":endLat,"lon":endLon,"type":"break"}],
            "costing":"auto",
            "id":"my_work_route"}

In [63]:
baseUrl = 'http://valhalla:8002/route'
route = requests.post(baseUrl, json=jsonDict)

In [101]:
print route.status_code
shape = route.json()['trip']['legs'][0]['shape']

200


In [97]:
json.dumps(jsonDict,separators=(',', ':'))

'{"locations":[{"lat":37.8261,"type":"break","lon":-122.25872},{"lat":37.790168,"type":"break","lon":-122.402264}],"costing":"auto","id":"my_work_route"}'

### 2. Pass Route Shape to Map Matching API 

In [81]:
payload = {
    "encoded_polyline": shape,
    "costing": "pedestrian",
    "directions_options": {
        "units": "kilometers"
    },
    "shape_match": "map_snap",
    "trace_options": {
        "turn_penalty_factor": 500
    }
}

In [82]:
baseUrl = 'http://valhalla:8002/trace_attributes?'

In [83]:
matched = requests.post(baseUrl, json=payload)

In [84]:
print 'Matched {0} points. Showing the first below: \n{1}'.format(
    len(matched.json()['matched_points']), matched.json()['matched_points'][0])

Matched 313 points. Showing the first below: 
{u'lon': -122.258926, u'distance_from_trace_point': 0.0, u'distance_along_edge': 0.119, u'type': u'matched', u'lat': 37.82618, u'edge_index': 0}


In [85]:
edges = matched.json()['edges']
print 'Got {0} edges. Showing the first below: \n {1}'.format(len(edges), edges[0])

Got 153 edges. Showing the first below: 
 {u'drive_on_right': True, u'max_downward_grade': 32768, u'end_heading': 206, u'surface': u'paved_smooth', u'names': [u'Manila Avenue'], u'end_shape_index': 1, u'way_id': 307501648, u'speed': 35, u'id': 294479583794, u'pedestrian_type': u'foot', u'use': u'road', u'density': 14, u'begin_shape_index': 0, u'max_upward_grade': 32768, u'traversability': u'both', u'lane_count': 0, u'travel_mode': u'pedestrian', u'mean_elevation': 32768, u'weighted_grade': 0.0, u'end_node': {u'elapsed_time': 116, u'type': u'street_intersection', u'time_zone': u'PST-08PDT+01,M3.2.0/02:00,M11.1.0/02:00', u'admin_index': 0}, u'road_class': u'residential', u'begin_heading': 206, u'bicycle_network': 0, u'traffic_segments': [{u'begin_percent': 0.0, u'starts_segment': True, u'ends_segment': True, u'end_percent': 1.0, u'segment_id': 324947008050}], u'length': 0.165}


### 3. Create Fake GPS traces

#### Extract coords from routes.shape, perturb, and save to dict

In [86]:
jsonDict = synthesize_gps(edges, shape, stddev=0.05)   # stddev in km

In [87]:
print 'Sending {0} fake GPS measurements to the reporter'.format(len(jsonDict['trace']))

Sending 154 fake GPS measurements to the reporter


### 4. Send Fake GPS to Reporter

In [88]:
baseUrl = 'http://reporter:8003/report'
report = requests.post(baseUrl, json=jsonDict)

In [89]:
segments = report.json()['segments']
print 'Reporter matched {0} GPS measurements to {1} OSMLR segments. Showing the first below: \n{2}'.format(
    len(jsonDict['trace']),len(segments), segments[3])

Reporter matched 154 GPS measurements to 104 OSMLR segments. Showing the first below: 
{u'segment_id': 157135771273, u'begin_shape_index': 0, u'start_time': -1.0, u'length': -1, u'end_time': 1495634936.505, u'end_shape_index': 3}


In [104]:
segments

[{u'begin_shape_index': 0,
  u'end_shape_index': 0,
  u'end_time': 1495634923.482,
  u'length': -1,
  u'segment_id': 788769921586,
  u'start_time': -1.0},
 {u'begin_shape_index': 0,
  u'end_shape_index': 0,
  u'end_time': 1495634924.247,
  u'length': 40,
  u'segment_id': 324913453618,
  u'start_time': 1495634923.482},
 {u'begin_shape_index': 0,
  u'end_shape_index': 0,
  u'end_time': 1495634925.967,
  u'length': 90,
  u'segment_id': 320517823026,
  u'start_time': 1495634924.247},
 {u'begin_shape_index': 0,
  u'end_shape_index': 3,
  u'end_time': 1495634936.505,
  u'length': -1,
  u'segment_id': 157135771273,
  u'start_time': -1.0},
 {u'begin_shape_index': 3,
  u'end_shape_index': 5,
  u'end_time': 1495634956.053,
  u'length': 441,
  u'segment_id': 205655479945,
  u'start_time': 1495634936.505},
 {u'begin_shape_index': 5,
  u'end_shape_index': 6,
  u'end_time': -1.0,
  u'length': -1,
  u'segment_id': 149116262025,
  u'start_time': 1495634956.568},
 {u'begin_shape_index': 6,
  u'end_shap

In [105]:
report.json().keys()

[u'segments', u'mode', u'provider']

In [108]:
dir(report)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__iter__',
 '__module__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']